In [1]:
import jetnet
from jetnet_dataloader import JetNetDataloader

In [2]:

import yaml
import os
p="g"
best_hparam="bestever_{}/hparams.yaml".format(p) #top
# best_hparam="/beegfs/desy/user/kaechben/fix_mask_scan3/lightning_logs/version_180/hparams.yaml" #top
config={'batch_size': 3096, 'bins': 5, 'coupling_layers': 15, 'dropout': 0.2, 'freq': 5, 'heads': 4, 'hidden': 500, 'max_epochs': 128, 
 'max_steps': 100000, 'n_dim': 3, 'n_part': 30, 'name': 'jupyter', 'parton': 'g', 'seed': 69, }

data_module = JetNetDataloader(config,) #this loads the data
data_module.setup("train")
data=data_module.data
mask=~(data[:,config["n_part"]*config["n_dim"]:].bool())

data=data[:,:config["n_part"]*config["n_dim"]].reshape(len(data),config["n_part"],config["n_dim"])


/home/kaechben/JetNet_NF/LitJetNet/LitNF/jetnet_dataloader.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(self.data).float()


In [5]:
from functools import partial
from performer_pytorch import SelfAttention
import torch
import math
import copy
import torch
import torch.nn as nn
from functools import partial
from numpy import random
@torch.jit.script
def delta_phi(a, b):
    return (a - b + math.pi) % (2 * math.pi) - math.pi


@torch.jit.script
def delta_r2(eta1, phi1, eta2, phi2):
    return (eta1 - eta2)**2 + delta_phi(phi1, phi2)**2


def to_pt2(x, eps=1e-8):
    pt2 = x[:, :2].square().sum(dim=1, keepdim=True)
    if eps is not None:
        pt2 = pt2.clamp(min=eps)
    return pt2


def to_m2(x, eps=1e-8):
    m2 = x[:, 3:4].square() - x[:, :3].square().sum(dim=1, keepdim=True)
    if eps is not None:
        m2 = m2.clamp(min=eps)
    return m2


def atan2(y, x):
    sx = torch.sign(x)
    sy = torch.sign(y)
    pi_part = (sy + sx * (sy ** 2 - 1)) * (sx - 1) * (-math.pi / 2)
    atan_part = torch.arctan(y / (x + (1 - sx ** 2))) * sx ** 2
    return atan_part + pi_part


def to_ptrapphim(x, return_mass=True, eps=1e-8, for_onnx=False):
    # x: (N, 4, ...), dim1 : (px, py, pz, E)
    px, py, pz, energy = x.split((1, 1, 1, 1), dim=1)
    pt = torch.sqrt(to_pt2(x, eps=eps))
    # rapidity = 0.5 * torch.log((energy + pz) / (energy - pz))
    rapidity = 0.5 * torch.log(1 + (2 * pz) / (energy - pz).clamp(min=1e-20))
    phi = (atan2 if for_onnx else torch.atan2)(py, px)
    if not return_mass:
        return torch.cat((pt, rapidity, phi), dim=1)
    else:
        m = torch.sqrt(to_m2(x, eps=eps))
        return torch.cat((pt, rapidity, phi, m), dim=1)


def boost(x, boostp4, eps=1e-8):
    # boost x to the rest frame of boostp4
    # x: (N, 4, ...), dim1 : (px, py, pz, E)
    p3 = -boostp4[:, :3] / boostp4[:, 3:].clamp(min=eps)
    b2 = p3.square().sum(dim=1, keepdim=True)
    gamma = (1 - b2).clamp(min=eps)**(-0.5)
    gamma2 = (gamma - 1) / b2
    gamma2.masked_fill_(b2 == 0, 0)
    bp = (x[:, :3] * p3).sum(dim=1, keepdim=True)
    v = x[:, :3] + gamma2 * bp * p3 + x[:, 3:] * gamma * p3
    return v


def p3_norm(p, eps=1e-8):
    return p[:, :3] / p[:, :3].norm(dim=1, keepdim=True).clamp(min=eps)


def pairwise_lv_fts(xi, xj, num_outputs=4, eps=1e-8, for_onnx=False):
    pti, rapi, phii = to_ptrapphim(xi, False, eps=None, for_onnx=for_onnx).split((1, 1, 1), dim=1)
    ptj, rapj, phij = to_ptrapphim(xj, False, eps=None, for_onnx=for_onnx).split((1, 1, 1), dim=1)

    delta = delta_r2(rapi, phii, rapj, phij).sqrt()
    lndelta = torch.log(delta.clamp(min=eps))
    if num_outputs == 1:
        return lndelta

    if num_outputs > 1:
        ptmin = ((pti <= ptj) * pti + (pti > ptj) * ptj) if for_onnx else torch.minimum(pti, ptj)
        lnkt = torch.log((ptmin * delta).clamp(min=eps))
        lnz = torch.log((ptmin / (pti + ptj).clamp(min=eps)).clamp(min=eps))
        outputs = [lnkt, lnz, lndelta]

    if num_outputs > 3:
        xij = xi + xj
        lnm2 = torch.log(to_m2(xij, eps=eps))
        outputs.append(lnm2)

    if num_outputs > 4:
        ei, ej = xi[:, 3:4], xj[:, 3:4]
        emin = ((ei <= ej) * ei + (ei > ej) * ej) if for_onnx else torch.minimum(ei, ej)
        lnet = torch.log((emin * delta).clamp(min=eps))
        lnze = torch.log((emin / (ei + ej).clamp(min=eps)).clamp(min=eps))
        outputs += [lnet, lnze]

    if num_outputs > 6:
        costheta = (p3_norm(xi, eps=eps) * p3_norm(xj, eps=eps)).sum(dim=1, keepdim=True)
        sintheta = (1 - costheta**2).clamp(min=0, max=1).sqrt()
        outputs += [costheta, sintheta]

    assert(len(outputs) == num_outputs)
    return torch.cat(outputs, dim=1)


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # From https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/layers/weight_init.py
    r"""Fills the input Tensor with values drawn from a truncated
    normal distribution. The values are effectively drawn from the
    normal distribution :math:`\mathcal{N}(\text{mean}, \text{std}^2)`
    with values outside :math:`[a, b]` redrawn until they are within
    the bounds. The method used for generating the random values works
    best when :math:`a \leq \text{mean} \leq b`.
    Args:
        tensor: an n-dimensional `torch.Tensor`
        mean: the mean of the normal distribution
        std: the standard deviation of the normal distribution
        a: the minimum cutoff value
        b: the maximum cutoff value
    Examples:
        >>> w = torch.empty(3, 5)
        >>> nn.init.trunc_normal_(w)
    """
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


class Embed(nn.Module):
    def __init__(self, input_dim, dims, normalize_input=True, activation='gelu'):
        super().__init__()

        self.input_bn = nn.BatchNorm1d(input_dim) if normalize_input else None
        module_list = []
        for dim in dims:
            module_list.extend([
                nn.LayerNorm(input_dim),
                nn.Linear(input_dim, dim),
                nn.GELU() if activation == 'gelu' else nn.ReLU(),
            ])
            input_dim = dim
        self.embed = nn.Sequential(*module_list)

    def forward(self, x):
        if self.input_bn is not None:
            # x: (batch, embed_dim, seq_len)
            x = self.input_bn(x)
            x = x.permute(2, 0, 1).contiguous()
        # x: (seq_len, batch, embed_dim)
        return self.embed(x)


class PairEmbed(nn.Module):
    def __init__(self, input_dim, dims, normalize_input=True, activation='gelu', eps=1e-8, for_onnx=False):
        super().__init__()

        self.for_onnx = for_onnx
        self.pairwise_lv_fts = partial(pairwise_lv_fts, num_outputs=input_dim, eps=eps, for_onnx=for_onnx)

        module_list = [nn.BatchNorm1d(input_dim)] if normalize_input else []
        for dim in dims:
            module_list.extend([
                nn.Conv1d(input_dim, dim, 1),
                nn.BatchNorm1d(dim),
                nn.GELU() if activation == 'gelu' else nn.ReLU(),
            ])
            input_dim = dim
        self.embed = nn.Sequential(*module_list)

        self.out_dim = dims[-1]

    def forward(self, x):
        # x: (batch, v_dim, seq_len)
        with torch.no_grad():
            batch_size, _, seq_len = x.size()
            if not self.for_onnx:
                i, j = torch.tril_indices(seq_len, seq_len, device=x.device)
                x = x.unsqueeze(-1).repeat(1, 1, 1, seq_len)
                xi = x[:, :, i, j]  # (batch, dim, seq_len*(seq_len+1)/2)
                xj = x[:, :, j, i]
                x = self.pairwise_lv_fts(xi, xj)
            else:
                x = self.pairwise_lv_fts(x.unsqueeze(-1), x.unsqueeze(-2)).view(batch_size, -1, seq_len * seq_len)

        elements = self.embed(x)  # (batch, embed_dim, num_elements)
        if not self.for_onnx:
            y = torch.zeros(batch_size, self.out_dim, seq_len, seq_len, dtype=elements.dtype, device=x.device)
            y[:, :, i, j] = elements
            y[:, :, j, i] = elements
        else:
            y = elements.view(batch_size, -1, seq_len, seq_len)
        return y


class Block(nn.Module):
    def __init__(self, embed_dim=128, num_heads=8, ffn_ratio=4,
                 dropout=0.1, attn_dropout=0.1, activation_dropout=0.1,
                 add_bias_kv=False, activation='gelu',# lin_attn=True,
                 scale_fc=False, scale_attn=False, scale_heads=False, scale_resids=False, ):
        super().__init__()

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.ffn_dim = embed_dim * ffn_ratio

        self.pre_attn_norm = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(
            embed_dim,
            num_heads,
            dropout=attn_dropout,
            add_bias_kv=add_bias_kv,
        )
#         self.attn=SelfAttention(dim=embed_dim,heads=num_heads,nb_features=self.ffn_dim,causal=False)#
        self.attn=nn.MultiheadAttention(
            embed_dim,
            num_heads,
            dropout=attn_dropout,
            add_bias_kv=add_bias_kv,
        )
        self.post_attn_norm = nn.LayerNorm(embed_dim) if scale_attn else None
        self.dropout = nn.Dropout(dropout)

        self.pre_fc_norm = nn.LayerNorm(embed_dim)
        self.fc1 = nn.Linear(embed_dim, self.ffn_dim)
        self.act = nn.GELU() if activation == 'gelu' else nn.ReLU()
        self.act_dropout = nn.Dropout(activation_dropout)
        self.post_fc_norm = nn.LayerNorm(self.ffn_dim) if scale_fc else None
        self.fc2 = nn.Linear(self.ffn_dim, embed_dim)
        #self.lin_attn=lin_attn
        self.c_attn = nn.Parameter(torch.ones(num_heads), requires_grad=True) if scale_heads else None
        self.w_resid = nn.Parameter(torch.ones(embed_dim), requires_grad=True) if scale_resids else None

    def forward(self, x, x_cls=None, padding_mask=None, attn_mask=None):
        """
        Args:
            x (Tensor): input to the layer of shape `(seq_len, batch, embed_dim)`
            x_cls (Tensor, optional): class token input to the layer of shape `(1, batch, embed_dim)`
            padding_mask (ByteTensor, optional): binary
                ByteTensor of shape `(batch, seq_len)` where padding
                elements are indicated by ``1``.
        Returns:
            encoded output of shape `(seq_len, batch, embed_dim)`
        """

        if x_cls is not None:
            with torch.no_grad():
                # prepend one element for x_cls: -> (batch, 1+seq_len)
                padding_mask = torch.cat((torch.zeros_like(padding_mask[:, :1]), padding_mask), dim=1)
            # class attention: https://arxiv.org/pdf/2103.17239.pdf
            residual = x_cls
            u = torch.cat((x_cls, x), dim=0)  # (seq_len+1, batch, embed_dim)
            u = self.pre_attn_norm(u)
           # if self.lin_attn:
            #    x = self.attn(x_cls, u, u, key_padding_mask=padding_mask)[0].unsqueeze(0) # (1, batch, embed_dim)
#            else:
            x = self.attn(x_cls, u, u, key_padding_mask=padding_mask)[0] # (1, batch, embed_dim)
        else:
            residual = x
            x = self.pre_attn_norm(x)
#             if self.lin_attn: 
#                 x = self.attn(x, x, x, key_padding_mask=padding_mask) #[0]#(seq_len, batch, embed_dim)  attn_mask=attn_mask
#             else:
            x = self.attn(x, x, x, key_padding_mask=padding_mask)[0] # (1, batch, embed_dim)

        if self.c_attn is not None:
            tgt_len, bsz = x.size(0), x.size(1)
            x = x.view(tgt_len, bsz, self.num_heads, self.head_dim)
            x = torch.einsum('tbhd,h->tbdh', x, self.c_attn)
            x = x.reshape(tgt_len, bsz, self.embed_dim)
        if self.post_attn_norm is not None:
            x = self.post_attn_norm(x)
        x = self.dropout(x)
        x += residual
        
        residual = x
        x = self.pre_fc_norm(x)
        x = self.act(self.fc1(x))
        x = self.act_dropout(x)
        if self.post_fc_norm is not None:
            x = self.post_fc_norm(x)
        x = self.fc2(x)
        x = self.dropout(x)
        if self.w_resid is not None:
            residual = torch.mul(self.w_resid, residual)
        x += residual
        return x


class ParticleTransformerCritic(nn.Module):

    def __init__(self,
                 input_dim,
                 num_classes=1,
                 # network configurations
                 pair_input_dim=3,
                 embed_dims=[128, 512, 128],
                 pair_embed_dims=[64, 64, 64],
                 num_heads=8,
                 num_layers=8,
                 num_cls_layers=2,
                 block_params=None,
                 cls_block_params={'dropout': 0, 'attn_dropout': 0, 'activation_dropout': 0},
                 fc_params=[],
                 activation='gelu',
                 # misc
                 trim=True,
                 for_inference=False,
                 use_amp=False,
                 **kwargs) -> None:
        super().__init__(**kwargs)

        self.trim = trim
        self.for_inference = for_inference
        self.use_amp = use_amp
        self._counter = 0

        default_cfg = dict(embed_dim=embed_dims[-1], num_heads=num_heads, ffn_ratio=4,
                           dropout=0.1, attn_dropout=0.1, activation_dropout=0.1,
                           add_bias_kv=False, activation=activation,
                           scale_fc=False, scale_attn=False, scale_heads=False, scale_resids=False)

        cfg_block = copy.deepcopy(default_cfg)
        if block_params is not None:
            cfg_block.update(block_params)
        

        cfg_cls_block = copy.deepcopy(default_cfg)
        if cls_block_params is not None:
            cfg_cls_block.update(cls_block_params)
        

        self.embed = Embed(input_dim, embed_dims, activation=activation)
        self.pair_embed = PairEmbed(
            pair_input_dim, pair_embed_dims + [cfg_block['num_heads']],
            for_onnx=for_inference) if pair_embed_dims is not None and pair_input_dim > 0 else None
        self.blocks = nn.ModuleList([Block(**cfg_block) for _ in range(num_layers)])
        self.cls_blocks = nn.ModuleList([Block(**cfg_cls_block) for _ in range(num_cls_layers)])
        self.norm = nn.LayerNorm(embed_dims[-1])

        if fc_params is not None:
            fcs = []
            in_dim = embed_dims[-1]
            for out_dim, drop_rate in fc_params:
                fcs.append(nn.Sequential(nn.Linear(in_dim, out_dim), nn.ReLU(), nn.Dropout(drop_rate)))
                in_dim = out_dim
            fcs.append(nn.Linear(in_dim, num_classes))
            self.fc = nn.Sequential(*fcs)
        else:
            self.fc = None

        # init
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dims[-1]), requires_grad=True)
        trunc_normal_(self.cls_token, std=.02)
        self.sig=torch.nn.Sigmoid()

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'cls_token', }

    def forward(self, x, v=None, mask=None):
        # x: (N, C, P)
        # v: (N, 4, P) [px,py,pz,energy]
        # mask: (N, 1, P) -- real particle = 1, padded = 0

        with torch.no_grad():
            if mask is None:
                mask = torch.ones_like(x[:, :1])
            mask = mask.bool()

            if self.trim and not self.for_inference:
                if self._counter < 5:
                    self._counter += 1
                else:
                    if self.training:
                        q = min(1, random.uniform(0.9, 1.02))
                        maxlen = torch.quantile(mask.type_as(x).sum(dim=-1), q).long()
                        rand = torch.rand_like(mask.type_as(x))
                        rand.masked_fill_(~mask, -1)
                        perm = rand.argsort(dim=-1, descending=True)
                        mask = torch.gather(mask, -1, perm)
                        x = torch.gather(x, -1, perm.expand_as(x))
                        if v is not None and self.pair_embed is not None:
                            v = torch.gather(v, -1, perm.expand_as(v))
                    else:
                        maxlen = mask.sum(dim=-1).max()
                    maxlen = max(maxlen, 1)
                    if maxlen < mask.size(-1):
                        mask = mask[:, :, :maxlen]
                        x = x[:, :, :maxlen]
                        if v is not None and self.pair_embed is not None:
                            v = v[:, :, :maxlen]

            padding_mask = ~mask.squeeze(1)  # (N, P)

        with torch.cuda.amp.autocast(enabled=self.use_amp):
            # input embedding
            x = self.embed(x).masked_fill(~mask.permute(2, 0, 1), 0)  # (P, N, C)
            attn_mask = None
            if v is not None and self.pair_embed is not None:
                attn_mask = self.pair_embed(v).view(-1, v.size(-1), v.size(-1))  # (N*num_heads, P, P)
            # transform
            for block in self.blocks:     
                x = block(x, x_cls=None, padding_mask=padding_mask, attn_mask=attn_mask)
            # extract class token
            cls_tokens = self.cls_token.expand(1, x.size(1), -1)  # (1, N, C)
            for block in self.cls_blocks:
                
                cls_tokens = block(x, x_cls=cls_tokens, padding_mask=padding_mask)
            x_cls = self.norm(cls_tokens).squeeze(0)
            # fc
            if self.fc is None:
                return x_cls
            output = self.fc(x_cls)
            if self.for_inference:
                output = self.sig(output)
            # print('output:\n', output)
            return output

class ParticleTransformerGen(nn.Module):

    def __init__(self,
                 input_dim,
                 out_dim=3,
                 # network configurations
                 fc_params=[],
                 pair_input_dim=3,
                 embed_dims=[128, 512, 128],
                 pair_embed_dims=[64, 64, 64],
                 num_heads=8,
                 num_layers=8,
                 activation='gelu',
                 # misc
                 trim=True,
                 for_inference=False,
                 use_amp=False,
                 **kwargs) -> None:
        super().__init__(**kwargs)

        self.trim = trim
        self.for_inference = for_inference
        self.use_amp = use_amp
        self._counter = 0

        default_cfg = dict(embed_dim=embed_dims[-1], num_heads=num_heads, ffn_ratio=4,
                           dropout=0.1, attn_dropout=0.1, activation_dropout=0.1,
                           add_bias_kv=False, activation=activation,
                           scale_fc=True, scale_attn=True, scale_heads=True, scale_resids=True)

        cfg_block = copy.deepcopy(default_cfg)
        self.embed = Embed(input_dim, embed_dims, activation=activation)
        self.pair_embed = PairEmbed(
            pair_input_dim, pair_embed_dims + [cfg_block['num_heads']],
            for_onnx=for_inference) if pair_embed_dims is not None and pair_input_dim > 0 else None
        self.blocks = nn.ModuleList([Block(**cfg_block) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(embed_dims[-1])
        
        fcs = []
        in_dim = embed_dims[-1]
        for out_dim, drop_rate in fc_params:
            fcs.append(nn.Sequential(nn.Linear(in_dim, out_dim), nn.ReLU(), nn.Dropout(drop_rate)))
            in_dim = out_dim
        fcs.append(nn.Linear(in_dim, out_dim))
        self.fc = nn.Sequential(*fcs)

    def forward(self, x, v=None, mask=None):
        # x: (N, C, P)
        # v: (N, 4, P) [px,py,pz,energy]
        # mask: (N, 1, P) -- real particle = 1, padded = 0

        with torch.no_grad():
            if mask is None:
                mask = torch.ones_like(x[:, :1])
            mask = mask.bool()

            # if self.trim and not self.for_inference:
            #     if self._counter < 5:
            #         self._counter += 1
            #     else:
            #         if self.training:
            #             q = min(1, random.uniform(0.9, 1.02))
            #             maxlen = torch.quantile(mask.type_as(x).sum(dim=-1), q).long()
            #             rand = torch.rand_like(mask.type_as(x))
            #             rand.masked_fill_(~mask, -1)
            #             perm = rand.argsort(dim=-1, descending=True)
            #             mask = torch.gather(mask, -1, perm)
            #             x = torch.gather(x, -1, perm.expand_as(x))
            #             if v is not None and self.pair_embed is not None:
            #                 v = torch.gather(v, -1, perm.expand_as(v))
            #         else:
            #             maxlen = mask.sum(dim=-1).max()
            #         maxlen = max(maxlen, 1)
            #         if maxlen < mask.size(-1):
            #             mask = mask[:, :, :maxlen]
            #             x = x[:, :, :maxlen]
            #             if v is not None and self.pair_embed is not None:
            #                 v = v[:, :, :maxlen]
            padding_mask = ~mask.squeeze(1)  # (N, P)

        with torch.cuda.amp.autocast(enabled=self.use_amp):
            # input embedding
            x = self.embed(x).masked_fill(~mask.permute(2, 0, 1), 0)  # (P, N, C)
            attn_mask = None
            if v is not None and self.pair_embed is not None:
                attn_mask = self.pair_embed(v).view(-1, v.size(-1), v.size(-1))  # (N*num_heads, P, P)
            # transform
            for block in self.blocks:
                x = block(x, x_cls=None, padding_mask=padding_mask, attn_mask=attn_mask)

            output = self.fc(x)
            # print('output:\n', output)
            return output


In [4]:
# from particle_transformer import ParticleTransformerCritic

model=ParticleTransformerCritic(3)
gen=ParticleTransformerGen(3)

p=data_module.data[:,:90].reshape(-1,30,3)
px=torch.cos(p[:,:,1])*p[:,:,2]
py=torch.sin(p[:,:,1])*p[:,:,2]
pz=torch.sinh(p[:,:,0])*p[:,:,2]
E=torch.sqrt(px**2+py**2+pz**2)
v=torch.stack((px,py,pz,E),dim=-1)


In [8]:
gen(p[:10].permute(0,2,1),v[:10].permute(0,2,1))

tensor([[[ 3.0381e+00,  4.9579e-01, -2.6674e+00],
         [ 1.5417e-01,  4.9677e-01, -2.7089e+00],
         [-1.0025e+00,  1.7906e+00, -2.2417e+00],
         [ 1.5954e+00,  1.6986e+00, -4.2189e+00],
         [ 1.4051e+00,  9.5534e-01, -2.4250e+00],
         [-6.6820e-01,  2.3693e-01, -3.4033e-01],
         [ 4.8175e-01,  1.0187e+00, -2.7466e+00],
         [ 7.9907e-01, -7.5628e-01, -1.1550e+00],
         [ 1.1858e+00,  2.1888e+00, -1.1491e+00],
         [-1.2118e+00,  1.8390e+00, -1.2990e+00]],

        [[ 1.7627e+00,  1.8285e+00, -2.4155e+00],
         [ 6.4342e-01, -7.5569e-01, -1.0874e+00],
         [-1.3481e+00,  1.8935e+00, -4.6243e-01],
         [ 7.8466e-01,  8.1616e-01, -3.1923e+00],
         [ 6.8084e-01,  3.4737e+00, -2.4908e+00],
         [-6.5216e-01, -3.2817e-01, -7.0682e-01],
         [ 6.2730e-01,  2.2780e+00, -3.0473e+00],
         [-6.3555e-01,  3.9729e-01,  3.4236e-01],
         [-5.6970e-01,  1.5619e+00, -8.9322e-01],
         [-1.6324e+00,  2.5417e+00, -2.7437e+00]

Calculating cov and mmd over 10 batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 19.46it/s]


(0.558, 0.02297494809004871)

In [31]:
import jetnet

class StandardScaler:
    def __init__(self, mean=None, std=None, epsilon=1e-7):
        """Standard Scaler.
        The class can be used to normalize PyTorch Tensors using native
        functions. The module does not expect the tensors to be of any specific shape;
         as long as the features are the last dimension in the tensor, the module
        will work fine.
        :param mean: The mean of the features. The property will be set after a call to fit.
        :param std: The standard deviation of the features. The property will be set after a call to fit.
        :param epsilon: Used to avoid a Division-By-Zero exception.
        """
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)

    def inverse_transform(self, values):
        return (values * self.std) + self.mean

    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)

    def to(self, dev):
        self.std = self.std.to(dev)
        self.mean = self.mean.to(dev)
        return self
test_set, _ = jetnet.datasets.JetNet.getData(
            jet_type="q",
            split="valid",
            num_particles=30,
            data_dir="/beegfs/desy/user/kaechben/datasets",
        )
 
data, _ = jetnet.datasets.JetNet.getData(
            jet_type="q",
            split="valid",
            num_particles=30,
            data_dir="/beegfs/desy/user/kaechben/datasets",
        )
    

test_set = torch.tensor(test_set)

data = torch.tensor(data)
data = torch.cat((data, test_set), dim=0)

# masks=np.sum(data.values[:,np.arange(3,120,4)],axis=1)
masks =data[:, :, -1].bool()
data = data[:, :, :-1]
n = masks.sum(axis=1)

masks = ~masks
data[masks, :] = (
    torch.normal(mean=torch.zeros_like(data[masks, :]), std=1).abs() * 1e-7
)

# standard scaling

scaler = StandardScaler()
data = scaler.fit_transform(data)
data = data.reshape(len(data), 90)
data = torch.tensor(
    torch.hstack(
        (data.reshape(len(data), 30 * 3), masks))
    
)
# self.data, self.test_set = train_test_split(self.data.cpu().numpy(), test_size=0.3)
test_set = data[-len(test_set) :].float()

/tmp/ipykernel_5193/675825127.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(


In [34]:
jetnet.evaluation.cov_mmd(scaler.inverse_transform(test_set[:,:90].reshape(-1,30,3)),scaler.inverse_transform(test_set[:,:90].reshape(-1,30,3)))

Calculating cov and mmd over 10 batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 18.23it/s]


(0.5389999999999999, 0.025488714683874404)